In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt

## Parameters

In [2]:
class Config:
    # Embeddings
    word_embedding_dim = 300
    char_embedding_dim = 100
    
    # RNN
    hidden_size_word = 300
    hidden_size_char = 100
    
    # Training parameters
    batch_size = 64
    num_epochs = 20
    display_every = 500
    evaluate_every = 1000
    num_checkpoints = 5
    decay_rate = 0.9
    
    learning_rate = 1.0
    decay_rate = 0.9
    
    # Testing parameters
    checkpoint_dir = ''
    
    UNK = "$UNK$"
    NUM = "$NUM$"
    NONE = "O"
    PAD = '$PAD$'

## Dataset 

Load annotated corpus for named entity recognition

https://www.kaggle.com/abhinavwalia95/entity-annotated-corpus

In [3]:
import nltk
import re
import os

class Dataset:
    def __init__(self):
        self.all_tags, self.all_words, self.all_chars = [], [], []
        
    def processing_word(self, word):
        word = word.lower()
        if word.isdigit():
            word = Config.NUM
        return word
        
    def load_dataset(self, path):
        words_col, tags_col = [], []
        with open(path) as f:
            words, tags = [], []
            for line in f:
                line = line.strip()
                if (len(line) == 0 or line.startswith("-DOCSTART-")):
                    if len(words) != 0:
                        words_col.append(' '.join(words))
                        tags_col.append(' '.join(tags))
                        words, tags = [], []
                else:
                    ls = line.split(' ')
                    word, tag = ls[0], ls[3]
                    word = self.processing_word(word)
                    
                    words.append(word)
                    tags.append(tag)
                    
                    self.all_words.append(word)
                    self.all_tags.append(tag)
                    self.all_chars.extend(list(word))
                    
                    
        return pd.DataFrame({'words': words_col, 'tags': tags_col})
        
    def download_and_load_datasets(self):
        self.all_tags, self.all_words = [], [] 
        
        dataset = tf.keras.utils.get_file(
          fname="CoNLL-2003.zip", 
          origin="https://s3.ap-northeast-2.amazonaws.com/bowbowbow-storage/dataset/CoNLL-2003.zip", 
          extract=True)
        
        dir_path = os.path.join(os.path.dirname(dataset), 'CoNLL-2003')
        train_df = self.load_dataset(os.path.join(dir_path, 'eng.train'))
        dev_df = self.load_dataset(os.path.join(dir_path, 'eng.testa'))
        test_df = self.load_dataset(os.path.join(dir_path, 'eng.testb'))
        return train_df, dev_df, test_df

dataset = Dataset()
train_df, dev_df, test_df = dataset.download_and_load_datasets()
train_df.head(10)

,tags,words
0,I-ORG O I-MISC O O O I-MISC O O,eu rejects german call to boycott british lamb .
1,I-PER I-PER,peter blackburn
2,I-LOC O,brussels 1996-08-22
3,O I-ORG I-ORG O O O O O O I-MISC O O O O O I-M...,the european commission said on thursday it di...
4,I-LOC O O O O I-ORG I-ORG O O O I-PER I-PER O ...,germany 's representative to the european unio...
5,O O O O O O O O O O O O O O O O O O O O I-ORG ...,""" we do n't support any such recommendation be..."
6,O O O O O O O O O O O O O O O O O O O O O O I-...,he said further scientific study was required ...
7,O O O O O O O I-ORG O O I-PER I-PER O O O O O ...,he said a proposal last month by eu farm commi...
8,I-PER O I-MISC O O O O I-LOC O I-LOC O O O O O...,fischler proposed eu-wide measures after repor...
9,O I-PER O O O O O O O I-ORG O O O O O O O O O ...,but fischler agreed to review his proposal aft...


In [4]:
word_list = list(set(dataset.all_words)) + [Config.PAD, Config.UNK]
word2idx = {w: i for i, w in enumerate(word_list)}
idx2word = {i: w for i, w in enumerate(word_list)}

tag_list = list(set(dataset.all_tags))
tag2idx = {w: i for i, w in enumerate(tag_list)}
idx2tag = {i: w for i, w in enumerate(tag_list)}

char_list = list(set(dataset.all_chars)) + [Config.PAD, Config.UNK]
char2idx = {w: i for i, w in enumerate(char_list)}
idx2char = {i: w for i, w in enumerate(char_list)}

## Model

In [5]:
class Model:
    def __init__(self, 
               num_classes, 
               vocab_size, 
               char_size,
               word_embedding_dim, 
               char_embedding_dim,
               hidden_size_word,
               hidden_size_char):
        
        self.word_ids = tf.placeholder(tf.int32, shape=[None, None], name='word_ids') 
        self.sequence_lengths = tf.placeholder(tf.int32, shape=[None], name="sequence_lengths")
        
        self.labels = tf.placeholder(tf.int32, shape=[None, None], name='labels')
        
        self.char_ids = tf.placeholder(tf.int32, shape=[None, None, None], name='char_ids') # [batch_size, max_sequence_length, max_word_length]
        self.word_lengths = tf.placeholder(tf.int32, shape=[None, None], name="word_lengths") # [batch_size, max_sequence_length]
        
        self.dropout = tf.placeholder(dtype=tf.float32, shape=[],name="dropout")
        
        initializer = tf.contrib.layers.xavier_initializer()
        
        # Word Embedding layer
        with tf.device('/cpu:0'), tf.variable_scope('word-embedding'):
            self._word_embeddings = tf.Variable(tf.random_uniform([vocab_size, word_embedding_dim], -0.25, 0.25), name='_word_embeddings', trainable=False)
            self.word_embeddings = tf.nn.embedding_lookup(self._word_embeddings, self.word_ids) # [batch_size, max_sequence_length, word_embedding_dim]
        
        # Char Embedding Layer
        with tf.variable_scope('char-embedding'):
            self._char_embeddings = tf.get_variable(dtype=tf.float32, shape=[char_size, char_embedding_dim], name='_char_embeddings')
            
            # [batch_size, max_sequence_length, max_word_length, char_embedding_dim]
            self.char_embeddings = tf.nn.embedding_lookup(self._char_embeddings, self.char_ids) 
            
            s = tf.shape(self.char_embeddings)
            
            # [batch_size*max_sequence_length, max_word_length, char_embedding_dim]
            char_embeddings = tf.reshape(self.char_embeddings, shape=[s[0]*s[1], s[2], char_embedding_dim])
            word_lengths = tf.reshape(self.word_lengths, shape=[-1])
            
            fw_cell = tf.nn.rnn_cell.LSTMCell(hidden_size_char, state_is_tuple=True)
            bw_cell = tf.nn.rnn_cell.LSTMCell(hidden_size_char, state_is_tuple=True)
            
            _, ((_, output_fw), (_, output_bw)) = tf.nn.bidirectional_dynamic_rnn(cell_fw=fw_cell, 
                                                                                   cell_bw=bw_cell, 
                                                                                   inputs=char_embeddings,
                                                                                   sequence_length=word_lengths,
                                                                                   dtype=tf.float32)
            # shape: [batch_size*max_sequnce_length, 2*hidden_size_char]
            output = tf.concat([output_fw, output_bw], axis=-1)
            output = tf.reshape(output, shape=[s[0], s[1], 2*hidden_size_char])
            
            # shape: # [batch_size, max_sequence_length, word_embedding_dim + 2*hidden_size_char]
            self.word_embeddings = tf.concat([self.word_embeddings, output], axis=-1) 
            self.word_embeddings = tf.nn.dropout(self.word_embeddings, self.dropout)
            
        # Bidirectional LSTM
        with tf.variable_scope("bi-lstm"):
            fw_cell = tf.nn.rnn_cell.LSTMCell(hidden_size_word)
            bw_cell = tf.nn.rnn_cell.LSTMCell(hidden_size_word)
            (output_fw, output_bw), _ = tf.nn.bidirectional_dynamic_rnn(cell_fw=fw_cell,
                                                                  cell_bw=bw_cell,
                                                                  inputs=self.word_embeddings,
                                                                  sequence_length= self.sequence_lengths, # [batch_size],
                                                                  dtype=tf.float32)
            
            self.rnn_outputs = tf.concat([output_fw, output_bw], axis=-1)  # [batch_size, max_sequence_length, 2*hidden_size_word]
            self.rnn_outputs = tf.nn.dropout(self.rnn_outputs, self.dropout)
        
        
        # Fully connected layer
        with tf.variable_scope('output'):
            self.W_output = tf.get_variable('W_output', shape=[2*hidden_size_word, num_classes],  dtype=tf.float32)
            self.b_output = tf.get_variable('b_output', shape=[num_classes], dtype=tf.float32, initializer=tf.zeros_initializer())
            
            nsteps = tf.shape(self.rnn_outputs)[1]
            rnn_outputs_flat = tf.reshape(self.rnn_outputs, [-1, 2*hidden_size_word])
            pred = tf.matmul(rnn_outputs_flat, self.W_output) + self.b_output
            
            self.logits = tf.reshape(pred, [-1, nsteps, num_classes]) # [batch_size, max_sequence_length, num_classes]
    
        # Calculate mean corss-entropy loss
        with tf.variable_scope('loss'):
            log_likelihood, trans_params = tf.contrib.crf.crf_log_likelihood(self.logits, self.labels, self.sequence_lengths)
            self.trans_params = trans_params  # need to evaluate it for decoding
            self.loss = tf.reduce_mean(-log_likelihood)
            
#             When CRF is not in use
#             self.losses = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=self.logits, labels=self.labels)
#             mask = tf.sequence_mask(self.sequence_lengths)
#             losses = tf.boolean_mask(self.losses, mask)
#             self.loss = tf.reduce_mean(losses) 
        
    # Length of the sequence data
    @staticmethod
    def _length(seq):
        relevant = tf.sign(tf.abs(seq))
        length = tf.reduce_sum(relevant, reduction_indices=1)
        length = tf.cast(length, tf.int32)
        return length
    
    @staticmethod
    def viterbi_decode(logits, trans_params):
        # get tag scores and transition params of CRF
        viterbi_sequences = []

        # iterate over the sentences because no batching in vitervi_decode
        for logit, sequence_length in zip(logits, sequence_lengths):
            logit = logit[:sequence_length]  # keep only the valid steps
            viterbi_seq, viterbi_score = tf.contrib.crf.viterbi_decode(
                logit, trans_params)
            viterbi_sequences += [viterbi_seq]

        return np.array(viterbi_sequences)

## Function for train

In [6]:
# Pre-trained glove
def load_glove(word_embedding_dim, word2idx):
    download_path = tf.keras.utils.get_file(
      fname="glove.6B.zip", 
      origin="http://nlp.stanford.edu/data/glove.6B.zip", 
      extract=True)
    
    embedding_path = os.path.join(os.path.dirname(download_path), 'glove.6B.300d.txt')
    print('embedding_path :', embedding_path)

    # initial matrix with random uniform
    initW = np.random.randn(len(word2idx), word_embedding_dim).astype(np.float32) / np.sqrt(len(word2idx))
    # load any vectors from the glove
    print("Load glove file {0}".format(embedding_path))
    f = open(embedding_path, 'r', encoding='utf8')
    for line in f:
        splitLine = line.split(' ')
        word = splitLine[0]
        embedding = np.asarray(splitLine[1:], dtype='float32')
        if word in word2idx:
            initW[word2idx[word]] = embedding
    return initW

def batch_iter(df, batch_size, num_epochs, shuffle=True, tqdm_disable=False):
    """
    Generates a batch iterator for a dataset.
    """
    data_size = len(df)
    num_batches_per_epoch = int((data_size - 1) / batch_size) + 1
    for epoch in tqdm(range(num_epochs), disable=tqdm_disable):
        # Shuffle the data at each epoch
        if shuffle:
            shuffle_indices = np.random.permutation(np.arange(data_size))
            shuffled_df= df.iloc[shuffle_indices]
        else:
            shuffled_df = df
        for batch_num in range(num_batches_per_epoch):
            start_index = batch_num * batch_size
            end_index = min((batch_num + 1) * batch_size, data_size)
            yield shuffled_df.iloc[start_index:end_index]    
            

def get_feed_dict(batch_df):
    max_length = max(map(lambda x : len(x.split(' ')), batch_df['words'].tolist()))
    
    max_length_word = 0
    for seq in batch_df['words'].tolist():
        for word in seq.split(' '):
            max_length_word = max(max_length_word, len(word))
    
    word_ids, sequence_lengths, labels, char_ids, word_lengths = [], [], [], [], []
    for index, row in batch_df.iterrows():
        sentence = row['words'].split(' ')
        tags = row['tags'].split(' ')

        word_ids_row, labels_row, char_ids_row, word_lengths_row = [], [], [], []
        for word in sentence:
            word_ids_row.append(word2idx[word])
        
            char_ids_row.append([char2idx[char] for char in word] + [char2idx[Config.PAD]]* (max_length_word - len(word)) )
            word_lengths_row.append(len(word))
        
        empty_char_ids = [char2idx[Config.PAD]]* max_length_word
        char_ids_row += [empty_char_ids] * (max_length - len(char_ids_row))
        word_lengths_row += [0] * (max_length - len(word_lengths_row))
        
        for tag in tags:
            labels_row.append(tag2idx[tag])

        if len(sentence) < max_length:
            word_ids_row += [word2idx[Config.PAD]]* (max_length - len(sentence))
            labels_row += [tag2idx[Config.NONE]]* (max_length - len(sentence))

        word_ids.append(word_ids_row)
        labels.append(labels_row)
        sequence_lengths.append(len(sentence))
        char_ids.append(char_ids_row)
        word_lengths.append(word_lengths_row)
    
    word_ids = np.array(word_ids)
    labels = np.array(labels)
    sequence_lengths = np.array(sequence_lengths)
    char_ids = np.array(char_ids)
    word_lengths = np.array(word_lengths)
    
    return word_ids, labels, sequence_lengths, char_ids, word_lengths

def evaluation(y, preds, lengths):
    from sklearn.metrics import classification_report
    arg_answer, arg_pred = [], []
    for i in range(len(y)):
        for j in range(lengths[i]):
            arg_answer.append(idx2tag[y[i][j]])
            arg_pred.append(idx2tag[preds[i][j]])

    print(classification_report(arg_answer, arg_pred))

## Train

In [7]:
import datetime
import time

from sklearn.metrics import f1_score
import warnings
import sklearn.exceptions
warnings.filterwarnings("ignore", category=sklearn.exceptions.UndefinedMetricWarning)

tf.reset_default_graph()
sess = tf.Session()
with sess.as_default():
    model = Model(
        num_classes=len(tag_list),
        vocab_size=len(word_list),
        char_size=len(char_list),
        word_embedding_dim=Config.word_embedding_dim,
        char_embedding_dim=Config.char_embedding_dim,
        hidden_size_word=Config.hidden_size_word,
        hidden_size_char=Config.hidden_size_char
    )
    
    global_step = tf.Variable(0, name='global_step', trainable=False)
    # train_op = tf.train.AdamOptimizer(Config.learning_rate).minimize(model.loss, global_step=global_step)
    
    optimizer = tf.train.AdadeltaOptimizer(Config.learning_rate, Config.decay_rate, 1e-6)
    gvs = optimizer.compute_gradients(model.loss)
    capped_gvs = [(tf.clip_by_value(grad, -1.0, 1.0), var) for grad, var in gvs]
    train_op = optimizer.apply_gradients(capped_gvs, global_step=global_step)
    
    # Output directory for models and summary
    timestamp = str(int(time.time()))
    out_dir = os.path.abspath(os.path.join(os.path.curdir, "30.runs", timestamp))
    print("Writing to {}\n".format(out_dir))

    sess.run(tf.global_variables_initializer())

    pretrain_W = load_glove(Config.word_embedding_dim, word2idx)
    sess.run(model._word_embeddings.assign(pretrain_W))
    print("Success to load pre-trained glove model!\n")
    
    # Generate batches
    batches = batch_iter(train_df, Config.batch_size, Config.num_epochs)
    for batch_df in batches:
        word_ids, labels, sequence_lengths, char_ids, word_lengths = get_feed_dict(batch_df)
        feed_dict = {
            model.word_ids: word_ids,
            model.labels: labels,
            model.sequence_lengths: sequence_lengths,
            model.char_ids: char_ids,
            model.word_lengths: word_lengths,
            model.dropout: 0.5,
        }
        _, step, loss, logits, trans_params = sess.run([
            train_op, global_step, model.loss, model.logits, model.trans_params], feed_dict)
        
        predictions = model.viterbi_decode(logits, trans_params)
        
        # Training log display
        if step % Config.display_every == 0:
            time_str = datetime.datetime.now().isoformat()
            print("{}: step {}, loss {:g}".format(time_str, step, loss))
            
        # Evaluation
        if step % Config.evaluate_every == 0:
            batches = batch_iter(dev_df, Config.batch_size, 1, tqdm_disable=True)
            
            total_loss, predictions_all, labels_all, sequence_lengths_all  = 0, [], [], []
            for batch_df in batches:
                word_ids, labels, sequence_lengths, char_ids, word_lengths = get_feed_dict(batch_df)
                feed_dict = {
                    model.word_ids: word_ids,
                    model.labels: labels,
                    model.sequence_lengths: sequence_lengths,
                    model.char_ids: char_ids,
                    model.word_lengths: word_lengths,
                    model.dropout: 1.0,
                }
                loss, logits, trans_params = sess.run([model.loss, model.logits, model.trans_params], feed_dict)
                predictions = model.viterbi_decode(logits, trans_params)
                
                total_loss += loss
                predictions_all += predictions.tolist()
                labels_all += labels.tolist()
                sequence_lengths_all += sequence_lengths.tolist()
        
            time_str = datetime.datetime.now().isoformat()
            print("\nDev Evaluation\n{}: loss {:g}\n".format(time_str, total_loss/len(predictions_all)))
            evaluation(labels_all, predictions_all, sequence_lengths_all)


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


/home/seungwon/project/tf-notes/venv/lib/python3.5/site-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Writing to /home/seungwon/project/tf-notes/30.runs/1559032651

embedding_path : /home/seungwon/.keras/datasets/glove.6B.300d.txt
Load glove file /home/seungwon/.keras/datasets/glove.6B.300d.txt


  0%|          | 0/20 [00:00<?, ?it/s]

Success to load pre-trained glove model!



...

 75%|███████▌  | 15/20 [08:30<02:51, 34.24s/it]

2019-05-28T17:47:12.183949: step 3500, loss 0.497705


 90%|█████████ | 18/20 [10:09<01:07, 33.51s/it]

2019-05-28T17:48:27.596385: step 4000, loss 0.47758

Dev Evaluation
2019-05-28T17:48:31.699727: loss 0.0142638

              precision    recall  f1-score   support

      B-MISC       0.33      0.25      0.29         4
       I-LOC       0.94      0.95      0.95      2094
      I-MISC       0.88      0.82      0.85      1264
       I-ORG       0.90      0.86      0.88      2092
       I-PER       0.97      0.98      0.97      3149
           O       0.99      0.99      0.99     42759

   micro avg       0.98      0.98      0.98     51362
   macro avg       0.84      0.81      0.82     51362
weighted avg       0.98      0.98      0.98     51362



100%|██████████| 20/20 [11:19<00:00, 34.19s/it]


In [8]:
batches = batch_iter(test_df, Config.batch_size, 1)
total_loss, predictions_all, labels_all, sequence_lengths_all  = 0, [], [], []
for batch_df in batches:
    word_ids, labels, sequence_lengths, char_ids, word_lengths = get_feed_dict(batch_df)
    feed_dict = {
        model.word_ids: word_ids,
        model.labels: labels,
        model.sequence_lengths: sequence_lengths,
        model.char_ids: char_ids,
        model.word_lengths: word_lengths,
        model.dropout: 1.0,
    }
    
    total_loss, logits, trans_params = sess.run([model.loss, model.logits, model.trans_params], feed_dict)
    predictions = model.viterbi_decode(logits, trans_params)
    
    total_loss += loss
    predictions_all += predictions.tolist()
    labels_all += labels.tolist()
    sequence_lengths_all += sequence_lengths.tolist()

time_str = datetime.datetime.now().isoformat()
print("\nTest Evaluation\n{}: loss {:g}\n".format(time_str, total_loss/len(predictions_all)))
evaluation(labels_all, predictions_all, sequence_lengths_all)

100%|██████████| 1/1 [00:04<00:00,  4.08s/it]



Test Evaluation
2019-05-28T17:49:35.939686: loss 0.000532494

              precision    recall  f1-score   support

       B-LOC       0.00      0.00      0.00         6
      B-MISC       1.00      0.11      0.20         9
       B-ORG       0.00      0.00      0.00         5
       I-LOC       0.90      0.90      0.90      1919
      I-MISC       0.78      0.78      0.78       909
       I-ORG       0.86      0.85      0.85      2491
       I-PER       0.97      0.96      0.97      2773
           O       0.99      0.99      0.99     38323

   micro avg       0.97      0.97      0.97     46435
   macro avg       0.69      0.57      0.59     46435
weighted avg       0.97      0.97      0.97     46435

